#### LeetCode - SQL - #196

Refer [196. Delete Duplicate Emails](https://leetcode.com/problems/delete-duplicate-emails/description/)

Write a solution to **delete** all duplicate emails, keeping only one unique email with the smallest `id`.

In [0]:
email_data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
email_columns = ['id', 'email']
email_schema = {'id':'int64', 'email':'object'}

In [0]:
import pandas

email_pandas_dataframe = pandas.DataFrame(data=email_data, columns=email_columns).astype(email_schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
email_dataframe = spark_context.createDataFrame(email_pandas_dataframe)
email_dataframe.printSchema()
email_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- email: string (nullable = true)

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
|  3|john@example.com|
+---+----------------+



#### Randomly removing rows based on column(s)

In [0]:
columns_to_check = ["email"]
email_dataframe_without_duplicates = email_dataframe.dropDuplicates(subset=columns_to_check)
email_dataframe_without_duplicates.show()

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+



#### Removing duplicate rows based on a column, keeping only one unique row with the smallest id.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy(email_dataframe.email).orderBy(email_dataframe.id)

email_dataframe = email_dataframe.withColumn("row_number", row_number().over(window_spec))
email_dataframe.show()

email_dataframe \
    .filter(email_dataframe.row_number == 1) \
    .orderBy(email_dataframe.id) \
    .drop(email_dataframe.row_number) \
    .show()

+---+----------------+----------+
| id|           email|row_number|
+---+----------------+----------+
|  2| bob@example.com|         1|
|  1|john@example.com|         1|
|  3|john@example.com|         2|
+---+----------------+----------+

+---+----------------+
| id|           email|
+---+----------------+
|  1|john@example.com|
|  2| bob@example.com|
+---+----------------+

